In [3]:
#Import modules
from __future__ import unicode_literals
import spacy
from spacy.lang.es import Spanish 
from spacy import displacy
import pandas as pd
import numpy as np
import json
from spacy.gold import docs_to_json
import random
import pickle

In [4]:
file = open("Files_Cleaning4/NER_data_Cleaned4.p", 'rb')
docs = pickle.load(file)

In [46]:
import re

In [22]:
for doc, context in docs:
    for ent in doc.ents:
        if ent.label_=="PER":
            try:
                start = ent.start - 5
            except:
                start = 0
            
            end=ent.end +5
            
            pre=doc[start:ent.start]
            post=doc[ent.end:end]
            if '\n' in pre.text:
                n=pre.find('\n')+1
                pre=pre[n:]
            if '\n' in post.text:
                n=post.find('\n')
                post=post[:n]
            if 'pintor' in pre.text:
                print(pre)
            if 'pintor' in post.text:
                print(post)

pintor vezino desta dicha cibdad
y Anton Sanchez, pintores
, pintores. Firma,
pintor vezino de Sevilla a
pintor... como su fiador
pintor... otorgo e conosco
pintor... otorgo e conozco
pintor... otorgo e conozco
pintor.., otorgo y
pintor... otorgo y conosco
pintor... otorgo e conosco
, pintor de imagineria[Firma:
pintor de imagineria[Firma: «
, pintores, vecinos de
, pintor de imagineria,
, pintor, vecino en
pintor vezino de esta dicha
pintor... en tal manera
pintor... nos anbos a
pintor e de anton sanches
jordan gonçales pintor e de
? asi mismo pintor (
, pintor de imagineria,
, pintor, y de
, pintor, y de
, pintor, vecino de
, pintor.
pintor... nos anbos a
, flamenco, pintor de
pintor... nos todos cuatro
, pintor de imagineria,
pintor... nos anbos a
, pintor, Juan Arias
Anton Perez, pintor,
pintor, Juan Arias y
, pintor de imagineria,
, pintor de imagineria,
pintor... otorgo e conozco
, pintor
pintor de imageneria vecino de
pintor vezino de esta cibdad
, pintor, vecino en
, pintor, ve

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'find'

In [40]:
for doc, context in docs:
    for ent in doc.ents:
        if ent.label_=="PER":
            try:
                start = ent.start - 1
                while (doc[start-1].label_ != "PER") and ('\n' not in doc[start:ent.start].text):
                    start = start -1
            except:
                pass
            print(doc[start:ent.start].text)

IndentationError: expected an indented block (<ipython-input-40-a713ead9d1a3>, line 11)

Testing some options, it seems the best plan of approach is to first extract information following a person ent, as usually these are the main descriptors (trade, neighborhood). 

We can then somehow mark this text as analyzed and look at the information to before the person ent.

In [57]:
PERdescriptors = []
for doc, context in docs:
    for ent in doc.ents:
        if ent.label_=='PER':
            try:
                end = ent.end +1
                while (doc[end+1].ent_type_ != 'PER') and ('\n' not in doc[ent.end:end+1].text) and (doc[end+1].pos_ != 'VERB') and (re.search(r'\.(?!\.\.)',doc[end+1].text)==None):
                    end = end +1
                if doc[ent.end:end].text is not "" and doc[ent.end:end].text is not "," and doc[ent.end:end].text is not "y" and doc[ent.end:end].text is not "...":
                    entry = [context['id'],ent.start,ent.end, ent.text, end, doc[ent.end:end].text]
                    PERdescriptors.append(entry)
            except:
                pass
            

In [59]:
len(PERdescriptors)

28459

In [60]:
uniquedescriptors = set([item[5] for item in PERdescriptors])

In [61]:
len(uniquedescriptors)

21200

As we can see, we have 28,459 descriptors for our PER entities. Of these, 21,200 are distinct. Having had a look at the descriptors, some have excess information that will have to be removed. Others contain several pieces of information, such as title/craft, location, role in the contract, age (for apprenticeships and perhaps masters examinations) and relationships. Sometimes these are separated by commas. 

There is also a special situation in which descriptors are plural - i.e., they describe multiple people. This would be the case when names are listed together. 

We will save all this information as a spreadsheet, and use Openrefine to structure it. 

On another note, we would also like to mark text that has already been analyzed/extracted somehow, perhaps with an extension attribute on its token. This would help us revise whichever text has not been analyzed towards the end of the creation of the database. For this reason, we marked the extent of the description. Tagged entities (once cleaned, as has already been done with PER tags) could also be marked as analyzed.

In [62]:
PERdescriptors_df = pd.DataFrame(PERdescriptors, columns=['docid','entstart','entend', 'entstr', 'descend', 'desctext'])

In [63]:
PERdescriptors_df

,docid,entstart,entend,entstr,descend,desctext
0,1,1,4,martin de gaynça,17,maestro mayor de las obras de canteria de la s...
1,1,18,22,juan sanchez de caliz,29,maestro mayor de las obras (roto
2,1,45,48,rodrigo de cordova,53,vecino de la ciudad de
3,1,69,72,luis de toro,132,en nombre de su majestad para fazer un muro de...
4,2,1,4,martin de gaynça,5,...
...,...,...,...,...,...,...
28454,8590,14,16,Juan Donaire,32,", maestro de cantería, vecino de la ciudad de ..."
28455,8591,4,8,PEDRO DE BORJA MACHADO,13,en nombre y en vos
28456,8591,14,16,Juan Donaire,32,", maestro de cantería, vecino de la ciudad de ..."
28457,8592,4,8,PEDRO DE BORJA MACHADO,13,en nombre y en vos


In [64]:
PERdescriptors_df.to_csv('PERDescriptors/PERdescriptors.csv')